# Reportes de exportaciones de firmas por año

Importar las librerías necesarias

In [14]:
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys

import pandas as pd
import numpy as np
from io import StringIO
import csv
import os
import time
import requests

## Firmas arándanos

### Año 2022

Guardar XPaths de los elementos del sitio web que son necesarios para la automatización

In [15]:
INPUT_FIELD_XPATH = "/html/body/center/form/table/tbody/tr[2]/td/input[1]" 
MENU_DESPLEGABLE_XPATH = '/html/body/center/form/table/tbody/tr[6]/td/select'
SEARCH_BUTTON_XPATH = "/html/body/center/form/center/input[1]"
DOWNLOAD_BUTTON_XPATH = "/html/body/form[1]/table[3]/tbody/tr/td/a[1]"
RETURN_BUTTON_XPATH = "/html/body/form[1]/table[3]/tbody/tr/td/a[3]"
ANIO_MENU_XPATH = "/html/body/center/form/table/tbody/tr[4]/td/select[1]"

Ejemplo de lista de RUCs extraída de las bases de datos públicas sobre exportaciones a nivel de firmas.

In [16]:
ruc_arandanos = [
    "20340584237",
    "20602822533",
    "20554556192",
    "20297939131",
    "20461642706",
    "20600807685",
    "20104902864",
    "20170040938",
    "20602842739",
    "20481121966",
    "20557530160",
    "20600529481",
    "20606009284",
    "20606133139",
    "20325117835",
    "20555837659",
    "20603778180",
    "20520866630",
    "20512441174",
    "20512217452",
    "20601438586",
    "20547669046",
    "20608170732",
    "20600235738",
    "20550858461",
    "20602266916",
    "20608345770",
    "20565586247",
    "20224133929",
    "20548245134",
    "20293718220",
]

### Año 2023

Configurar las opciones del driver para esta página en específico. 

* Se define una ruta de descarga específica para que todos los reportes se guarden en el mismo lugar.
* Se deshabilita la ventana emergente de `"Guardar como..."` (`"download.prompt_for_download": False`). Esto permite que el script pueda descargar cientos de archivos sin detenerse a pedir confirmación.
* El portal de Aduanet es `HTTP`. Se añaden argumentos (`--allow-running-insecure-content`) para asegurar que Chrome confíe en el sitio y no bloquee la descarga de los archivos Excel por considerarlos "inseguros". Además, sirve para lidiar con el pop up que aparece automáticamente cuando se intenta ingresar a la página.

Finalmente, el script inicia el navegador con estas opciones aplicadas y abre la URL del portal de Aduanet.

In [17]:
chrome_options = Options()

chrome_options.add_argument("--allow-running-insecure-content")  # Allow insecure content
chrome_options.add_argument("--unsafely-treat-insecure-origin-as-secure=http://www.aduanet.gob.pe/cl-ad-itconsultadwh/ieITS01Alias?accion=consultar&CG_consulta=1")  # Replace example.com with your site's domain
chrome_options.add_experimental_option("prefs", {
    "download.default_directory": r'/Users/andreaquispe/Library/CloudStorage/GoogleDrive-a.e.q.maslucan@gmail.com/My Drive/UNIVERSIDAD/2025-2/Seminario/Entrega 3/Bases_datos/Ejemplo',
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True
})

driver = webdriver.Chrome( options = chrome_options )
url = 'http://www.aduanet.gob.pe/cl-ad-itconsultadwh/ieITS01Alias?accion=consultar&CG_consulta=1'
driver.get(url)
driver.maximize_window()

In [18]:
print(f"Iniciando proceso de scraping para {len(ruc_arandanos)} RUCs.")

for i, ruc in enumerate(ruc_arandanos):
    print(f"\n--- Procesando RUC {i+1}/{len(ruc_arandanos)}: {ruc} ---")

    try:
        # 1. LOCALIZAR CAMPO DE INPUT
        input_text_fname = driver.find_element(By.XPATH, INPUT_FIELD_XPATH)
        
        # *CRUCIAL*: Borrar el contenido anterior antes de ingresar el nuevo RUC
        input_text_fname.clear() 
        
        # 2. INGRESAR EL RUC ACTUAL
        input_text_fname.send_keys(ruc)
        print(f"   -> RUC ingresado: {ruc}")
        
        # 3. LOCALIZAR MENÚ DESPLEGABLE DE TIPO DE CONSULTA y SELECCIONAR OPCIÓN
        # Es mejor localizar el elemento en cada iteración para evitar errores de referencia
        select_element = driver.find_element(By.XPATH, MENU_DESPLEGABLE_XPATH)
        drop = Select(select_element)
        drop.select_by_value("40") # '40' es el valor de la opción
        print("   -> Opción de Tipo de Consulta '40' seleccionada.")
        time.sleep(2)

        # 3.1. LOCALIZAR MENÚ DESPLEGABLE DE AÑO y SELECCIONAR OPCIÓN
        select_anio = driver.find_element(By.XPATH, ANIO_MENU_XPATH)
        drop_anio = Select(select_anio)
        drop_anio.select_by_value("31") # '31' es el valor de la opción para el año
        print("   -> Opción de Año '31' seleccionada.")
        time.sleep(2)

        # 4. CLIC EN EL BOTÓN DE BÚSQUEDA
        buscar = driver.find_element(By.XPATH, SEARCH_BUTTON_XPATH)
        buscar.click()
        print("   -> Búsqueda iniciada.")
        time.sleep(5) # Esperar más tiempo por si la búsqueda tarda

        # 5. CLIC EN EL BOTÓN DE DESCARGA
        descarga = driver.find_element(By.XPATH, DOWNLOAD_BUTTON_XPATH)
        descarga.click()
        print("   -> Descarga iniciada.")
        # Esperamos un tiempo prudente para que la descarga se complete
        time.sleep(6) 

        # 6. CLIC EN EL BOTÓN DE REGRESAR
        regresar = driver.find_element(By.XPATH, RETURN_BUTTON_XPATH)
        regresar.click()
        print("   -> Regresando a la página de búsqueda.")
        time.sleep(4)
        
    except Exception as e:
        # Manejo de errores: si la página da un error o un elemento no se encuentra
        print(f"!!! Error al procesar el RUC {ruc}. Saltando al siguiente. Error: {e}")
        # Intenta regresar a la página principal para limpiar el estado y continuar
        try:
             driver.get(url) 
             time.sleep(3)
        except:
             # Si no puede regresar, el script terminará, pero al menos intentó un recovery.
             pass 

print("\nProceso de scraping completado. Cerrando navegador.")
driver.quit()

Iniciando proceso de scraping para 31 RUCs.

--- Procesando RUC 1/31: 20340584237 ---
   -> RUC ingresado: 20340584237
   -> Opción de Tipo de Consulta '40' seleccionada.
   -> Opción de Año '31' seleccionada.
   -> Búsqueda iniciada.
   -> Descarga iniciada.
   -> Regresando a la página de búsqueda.

--- Procesando RUC 2/31: 20602822533 ---
   -> RUC ingresado: 20602822533
   -> Opción de Tipo de Consulta '40' seleccionada.
   -> Opción de Año '31' seleccionada.
   -> Búsqueda iniciada.
   -> Descarga iniciada.
   -> Regresando a la página de búsqueda.

--- Procesando RUC 3/31: 20554556192 ---
   -> RUC ingresado: 20554556192
   -> Opción de Tipo de Consulta '40' seleccionada.
   -> Opción de Año '31' seleccionada.
   -> Búsqueda iniciada.
   -> Descarga iniciada.
   -> Regresando a la página de búsqueda.

--- Procesando RUC 4/31: 20297939131 ---
   -> RUC ingresado: 20297939131
   -> Opción de Tipo de Consulta '40' seleccionada.
   -> Opción de Año '31' seleccionada.
   -> Búsqueda i